In [1]:
# imports
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import hf_token

In [2]:
# load dnd handbook
loader = PyPDFLoader(r'C:\Users\connor\PycharmProjects\portfolio\rag\data\DnD_BasicRules_2018.pdf')
dnd_doc = loader.load()

# test
# loader = PyPDFLoader(r'C:\Users\connor\PycharmProjects\llm\data\test.pdf')
# dnd_doc = loader.load()
dnd_doc

[Document(metadata={'source': 'C:\\Users\\connor\\PycharmProjects\\portfolio\\rag\\data\\DnD_BasicRules_2018.pdf', 'page': 0}, page_content='1D&D Basic Rules, Version 1.0, Released November 2018\nCredits \nLead Designers:  Jeremy Crawford, Mike Mearls, \nChristopher Perkins\nDesign Team:  James Wyatt, Rodney Thompson, Robert J. \nSchwalb, Peter Lee, Steve Townshend, Bruce R. Cordell\nManaging Editor:  Jeremy Crawford\nEditing Team:  Chris Sims, Michele Carter, Scott Fitzgerald Gray, \nKim Mohan\nGraphic Designers:  Bree Heiss, Emi Tanji\nInterior Illustrator: Jaime Jones, Richard Whitters\nAdditional Contributors:  Matt Sernett, Chris Dupuis, Tom \nLaPille, Chris Tulach, Miranda Horner, Jennifer Clarke Wilkes, \nSteve Winter, Chris Youngs, Ben Petrisor, Tom Olsen\nProducer:  Greg Bilsland\nProject Management:  Neil Shinkle, Kim Graham, John Hay\nBrand and Marketing:  Nathan Stewart, Liz Schuh, Chris Lindsay, \nShelly Mazzanoble, Hilary Ross, Laura Tommervik, Kim \nLundstrom, Trevor Kid

In [3]:
# split text into chunks
# Create an instance of the recursive splitter class
splitter = RecursiveCharacterTextSplitter(
    chunk_size=24,
    chunk_overlap=10
)

# Split the string and print the chunks
docs = splitter.split_documents(dnd_doc)

In [4]:
# max batch size for Chroma. Fix batching later
docs = docs[:5460]

In [ ]:
# Embed the documents in a persistent Chroma vector database

model_id = "sentence-transformers/all-MiniLM-L6-v2"
# update to gpu later. It's involved
model_kwargs = {'device':'cpu'}
embedding_function = HuggingFaceEmbeddings(model_name=model_id, model_kwargs=model_kwargs)
data_path = r'C:\Users\connor\PycharmProjects\llm\data'

# this has issues if you try to update it. Rerunning the code seems to just cause the notebook to spin endlessly and it will never update the contents of the persisted ChromaDB even if you change the above code. Only deleting the whole db and all chroma files and rerunning seems to cause it to work. 
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory=data_path
)

C:\Users\connor\AppData\Local\Programs\Python\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\connor\AppData\Local\Programs\Python\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
data_path = r'C:\Users\connor\PycharmProjects\portfolio\rag\data'
model_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding_function = HuggingFaceEmbeddings(model_name=model_id)
vectordb = Chroma(persist_directory=data_path, embedding_function=embedding_function)

retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
vectordb.get()

In [ ]:
# Add placeholders to the message string
message = """
Answer the following question using primarily the context provided:

Context:
{context}

Question:
{question}

Answer:
"""

# Create a chat prompt template from the message string
prompt_template = ChatPromptTemplate.from_messages([("human", message)])

In [ ]:
llm = HuggingFaceEndpoint(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token)

In [ ]:
# rag chain
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
             | prompt_template
             | llm)

In [ ]:
response = rag_chain.invoke("Under the Using these Rules section, what is part 2?")
print(response)